In [44]:
import pandas as pd
import numpy as np
from collections import Counter

In [56]:
# Import packages
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV


%matplotlib inline

g:\installed\pyenvironments\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [22]:
customer_final_data = pd.read_excel('Data/customer_final_data.xlsx')
transaction_final_data = pd.read_excel('Data/Transaction_final_data.xlsx')

In [32]:
customer_final_data = customer_final_data.drop(columns=['Unnamed: 0'],axis=1)

In [33]:
transaction_final_data = transaction_final_data.drop(columns=['Unnamed: 0'],axis=1)

In [38]:
customer_final_data = customer_final_data.rename(columns={customer_final_data.columns[0]: 'Row_ID'})

In [41]:
print(customer_final_data.shape)
customer_final_data.head(20)

(36386, 25)


,Row_ID,Coverage_Priority__c,ZI_Company_Type__c,count,ZI_Number_Of_Locations__c,cumulative_sum_revenue,Platinum_Gold_Quality_Ratio,tenure,AvgCost,AvgQuantity,...,Family_Industry_Description_New__c,Territory__c,compliance_score,Gradient_Revenue,time_between_sales_trend,item_diversity_index,Client_revenue,plat_gold_per_transaction,churn_standard,Tier__c
0,0,Uncovered,Unknown,2,2,0.00,5000.000000,13,0.000000,0.000000,...,Unassigned,WEST,Low,-517.54,0,1.000000,other,0.500000,1,TIER8
1,1,Inside Sales,PRIVATE,8,5,1308.98,1538.461538,826,163.622500,4.375000,...,Chemical,MIDWEST,High,159.70,-41,1.625000,$10mil-$100mil,0.250000,1,TIER7
2,2,BDM,PUBLIC,5,296,1005.31,0.000000,638,201.062000,10.600000,...,Pharma & Healthcare,MIDWEST,High,864.35,609,2.600000,Over $5bil,0.000000,1,TIER2-TrailBlazer
3,3,Uncovered,Unknown,63,2,32307.39,1092.896175,2075,512.815714,78.634921,...,Mining Oil & Gas,Mid-Atlantic,High,-1221.77,2,2.857143,other,0.317460,0,TIER5-Scout
4,4,BDM,PRIVATE,49,16,16894.67,3302.752294,2015,344.789184,17.714286,...,Chemical,WEST,High,353.51,212,2.183673,other,0.734694,1,TIER3-Pioneer
5,5,BDM,PUBLIC,19,51,20982.62,2727.272727,1476,1104.348421,314.631579,...,Chemical,SOUTHEAST,High,-1357.43,-123,1.157895,Over $5bil,0.315789,1,TIER2-TrailBlazer
6,6,Uncovered,PRIVATE,59,8,35532.33,1034.482759,2107,602.242881,20.423729,...,Mining Oil & Gas,WEST,Low,-904.93,877,1.440678,$100mil-$500mil,0.152542,0,TIER5-Scout
7,7,BDM,PRIVATE,111,2,31955.25,2010.309278,2165,287.885135,15.288288,...,Pharma & Healthcare,SOUTHEAST,High,-106.71,89,1.747748,$10mil-$100mil,0.351351,0,TIER6
8,8,Uncovered,PRIVATE,11,6,1156.57,2352.941176,1672,105.142727,4.909091,...,Chemical,UNKNOWN,Low,-127.85,31,1.545455,$100mil-$500mil,0.363636,0,TIER8
9,9,Inside Sales,PUBLIC,31,2243,19821.87,233.644860,2127,639.415161,27.161290,...,Mining Oil & Gas,WEST,High,417.56,114,6.741935,Over $5bil,0.161290,0,TIER2-TrailBlazer


In [40]:
transaction_final_data.shape

(418280, 25)

In [46]:
print(Counter(customer_final_data['churn_standard']))

Counter({1: 19285, 0: 17101})


In [47]:
print(Counter(transaction_final_data['churn_standard']))

Counter({0: 378014, 1: 40266})


In [97]:
print(customer_final_data['churn_standard'].value_counts())
print('\nTotal Churn Rate: {:.2%}'.format(customer_final_data[customer_final_data['churn_standard'] == 1].shape[0] / customer_final_data.shape[0]))

1    19285
0    17101
Name: churn_standard, dtype: int64

Total Churn Rate: 53.00%


In [98]:
# unique values for each column containing a categorical feature
def unique_values():
    cat_columns = np.unique(customer_final_data.select_dtypes('object').columns)
    for i in cat_columns:
        print(i, customer_final_data[i].unique())

unique_values()

Client_revenue ['other' '$10mil-$100mil' 'Over $5bil' '$100mil-$500mil' '$1mil-$10mil'
 '$500mil-$5bil' 'Under $1mil']
Coverage_Priority__c ['Uncovered' 'Inside Sales' 'BDM' 'Government' 'Government Contractor'
 'Distributor' 'Spacemaster' 'Internal Customer' 'Strategic Distributor'
 'Amazon']
Family_Industry_Description_New__c ['Unassigned' 'Chemical' 'Pharma & Healthcare' 'Mining Oil & Gas'
 'Transport & Warehouse' 'Other Manufacturing' 'Gov & Military'
 'Agriculture & Food' 'Wholesale Trade' 'Waste Management' 'Automotive'
 'Education' 'Aviation & Aerospace' 'Retail Trade' 'Transport Equipment'
 'Services' 'Other' 'Technology']
Territory__c ['WEST' 'MIDWEST' 'Mid-Atlantic' 'SOUTHEAST' 'UNKNOWN' 'NORTHEAST'
 'GREAT LAKES' 'CANADA']
Tier__c ['TIER8' 'TIER7' 'TIER2-TrailBlazer' 'TIER5-Scout' 'TIER3-Pioneer' 'TIER6'
 'TIER1-Commander' 'GOVCON' 'TIER4-Explorer' 'SM3' 'GOV' 'SM1' 'SM2'
 'DISTST' nan]
ZI_Company_Type__c ['Unknown' 'PRIVATE' 'PUBLIC' 'GOVERNMENT' 'NPO' 'EDUCATION' 'OTHER'
 

In [99]:
# list of binary variables, except 'Churn'
bin_var = [col for col in customer_final_data.columns if len(customer_final_data[col].unique()) == 2 and col != 'Churn']

# list of categorical variables
cat_var = [col for col in customer_final_data.select_dtypes(['object']).columns.tolist() if col not in bin_var]

In [100]:
print(bin_var)

['churn_standard']


In [101]:
print(cat_var)

['Coverage_Priority__c', 'ZI_Company_Type__c', 'Family_Industry_Description_New__c', 'Territory__c', 'compliance_score', 'Client_revenue', 'Tier__c']


In [102]:
# apply Label Encoding for binaries
le = LabelEncoder()
for col in bin_var:
    customer_final_data[col] = le.fit_transform(customer_final_data[col])

# apply get_dummies for categorical
df = pd.get_dummies(customer_final_data, columns=cat_var)

df.head()

,Row_ID,count,ZI_Number_Of_Locations__c,cumulative_sum_revenue,Platinum_Gold_Quality_Ratio,tenure,AvgCost,AvgQuantity,Popular_item_count,No.Of Managers,...,Tier__c_SM2,Tier__c_SM3,Tier__c_TIER1-Commander,Tier__c_TIER2-TrailBlazer,Tier__c_TIER3-Pioneer,Tier__c_TIER4-Explorer,Tier__c_TIER5-Scout,Tier__c_TIER6,Tier__c_TIER7,Tier__c_TIER8
0,0,2,2,0.00,5000.000000,13,0.000000,0.000000,1,0.0,...,0,0,0,0,0,0,0,0,0,1
1,1,8,5,1308.98,1538.461538,826,163.622500,4.375000,4,1.0,...,0,0,0,0,0,0,0,0,1,0
2,2,5,296,1005.31,0.000000,638,201.062000,10.600000,2,20.0,...,0,0,0,1,0,0,0,0,0,0
3,3,63,2,32307.39,1092.896175,2075,512.815714,78.634921,6,5.0,...,0,0,0,0,0,0,1,0,0,0
4,4,49,16,16894.67,3302.752294,2015,344.789184,17.714286,6,2.0,...,0,0,0,0,1,0,0,0,0,0


## Splitting dataset

In [103]:
#from fast_ml.model_development import train_valid_test_split

#X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(customer_final_data, target = 'churn_standard',train_size=0.8, valid_size=0.1, test_size=0.1)

In [104]:
# feature matrix
X = df.drop('churn_standard', axis=1)
# target vector
y = df['churn_standard']

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [105]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

In [106]:
rus = RandomUnderSampler()
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [ ]:
#check_df = pd.DataFrame(y_train_rus)
#np.isnan(np.min(X_train_rus))

In [70]:
#check_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25732 entries, 0 to 25731
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   churn_standard  25732 non-null  int64
dtypes: int64(1)
memory usage: 201.2 KB


In [ ]:
svc = SVC(gamma='auto')
lr = LogisticRegression()
xgb = XGBClassifier()

model = []
cross_val = []
recall = []
for i in (svc, lr, xgb):
    model.append(i.__class__.__name__)
    cross_val.append(cross_validate(i, X_train_rus, y_train_rus, scoring='recall'))

for d in range(len(cross_val)):
    recall.append(cross_val[d]['test_score'].mean())


In [108]:
model_recall = pd.DataFrame
pd.DataFrame(data=recall, index=model, columns=['Recall'])


,Recall
SVC,NaN
LogisticRegression,NaN
XGBClassifier,0.957752


In [ ]:
# XGBoost
# parameter to be searched
param_grid = {'n_estimators': range(0,1000,25)}

# find the best parameter
grid_search = GridSearchCV(xgb, param_grid, scoring='recall')
grid_result = grid_search.fit(X_train_rus, y_train_rus)

print(f'Best result: {grid_result.best_score_} for {grid_result.best_params_}')

In [ ]:
# XGBoost
xgb = XGBClassifier(n_estimators=150) #use no of estimators from above

# parameters to be searched
param_grid = {'max_depth': range(1,8,1),
              'min_child_weight': np.arange(0.0001, 0.5, 0.001)}

# find the best parameters
grid_search = GridSearchCV(xgb, param_grid, scoring='recall', n_jobs=-1)
grid_result = grid_search.fit(X_train_rus, y_train_rus)

print(f'Best result: {grid_result.best_score_} for {grid_result.best_params_}')

In [ ]:
# XGBoost
xgb = XGBClassifier(n_estimators=150, max_depth=1, min_child_weight=0.0001) #use child_weight from above

# parameter to be searched
param_grid = {'gama': np.arange(0.0,20.0,0.05)}

# find the best parameters
grid_search = GridSearchCV(xgb, param_grid, scoring='recall', n_jobs=-1)
grid_result = grid_search.fit(X_train_rus, y_train_rus)

print(f'Best result: {grid_result.best_score_} for {grid_search.best_params_}')

In [ ]:
# XGBoost
xgb = XGBClassifier(n_estimators=150, max_depth=1, min_child_weight=0.0001, gama=0.0) #use feature from above

# parameter to be searched
param_grid = {'learning_rate': [0.0001, 0.01, 0.1, 1]}

# find the best parameter
grid_search = GridSearchCV(xgb, param_grid, scoring='recall')
grid_result = grid_search.fit(X_train_rus, y_train_rus)

print(f'Best result: {grid_search.best_score_} for {grid_search.best_params_}')

In [ ]:
# final XGBoost model
xgb = XGBClassifier(learning_rate=0.0001, n_estimators=150, max_depth=1, min_child_weight=0.0001, gamma=0) #use learning rate from above
xgb.fit(X_train_rus, y_train_rus)

# prediction
X_test_xgb = scaler.transform(X_test)
y_pred_xgb = xgb.predict(X_test_xgb)

# classification report
print(classification_report(y_test, y_pred_xgb))

# confusion matrix
fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(y_test, y_pred_xgb, normalize='true'), annot=True, ax=ax)
ax.set_title('Confusion Matrix')
ax.set_ylabel('Real Value')
ax.set_xlabel('Predicted Value')

plt.show()